In [1]:
"""Errors might remain if Kernel is not restarted"""
"""Changes to local modules might not update if Kernel is not restarted"""

'Changes to local modules might not update if Kernel is not restarted'

In [2]:
import numpy as np
# provide point coordinates/sets of points, 
# can be skipped if you already have a Euclidean distance matrix corresponding to your sets
M = 3
N = 5
X = np.random.uniform(size = (M, 4))
Y = np.random.uniform(size = (N, 3))

In [3]:
# provide a proximity set matrix for between the points of X and Y
# f cannot be zero everywhere
fXY = np.random.uniform(low = 0, high = 2, size = (M, N))

In [4]:
import scipy as sp
# compute distance/proximity within sets of points (must be Euclidean distance matrices) satisfying (ii) of Theorem 2.1
DX = sp.spatial.distance.pdist(X)
DX = np.max(fXY) + sp.spatial.distance.squareform(DX) # added max(fXY) to satisfy (ii)
DY = sp.spatial.distance.pdist(Y)
DY = np.max(fXY) + sp.spatial.distance.squareform(DY) # added max(fXY) to satisfy (ii)

In [5]:
# Optional: provide a distance vectors for distance to the origin satisfying condition (ii) of Theorem 2.1
UX = np.random.uniform(low = np.max(fXY), high=np.max(fXY) + 0.001*np.std(fXY), size = M)
UY = np.random.uniform(low = np.max(fXY), high=np.max(fXY) + 0.001*np.std(fXY), size = N)

In [12]:
# Provide parameters (c1, c2, c3) as dictionary, using conditions (i), (ii), and (iii) of Theorem 3.1 as a guideline
c1, c2 = 1/2, 2
a = 1. - 1./(M+N)
b = 2.*c2/(M+N)
c3 =  min(((2.*c1 + c2) - b)/a, 2*c2+2)
c = {"c1":c1, "c2":c2, "c3":c3} 
print(c1, c2, c3) # make sure they are all positive

0.5 2 2.857142857142857


In [13]:
# Compute cosine law matrix
import Methods.CosLM as CosLM
COS_MAT, c1, c2, c3, zeta_f = CosLM.CosLM(DX, DY, UX, UY, fXY, c) # w1 = x1
print("Is the cosine law matrix is symmetric:", np.all(np.isclose(COS_MAT,COS_MAT.T)))
sigma, U = sp.linalg.eigh(COS_MAT)
sigma = np.real(sigma) # if COS_MAT is symmetric, thus imaginary number are supposed to be zero or numerical zeros
sigma[np.isclose(sigma, np.zeros(len(sigma)))] = 0
print(sigma[np.argsort(sigma)[::-1]])

Is the cosine law matrix is symmetric: True
[159.13570831  40.52350656  39.59256908  38.87330252  38.24142739
  37.24236032  36.59562013  27.53916811  11.05039515   0.        ]


In [14]:
print("Is COS_MAT PSD:", np.all(sigma>=0))
if (not np.all(sigma>=0)):
    print("Check the conditions in Theorem 3.1")

Is COS_MAT PSD: True


In [15]:
### check distances and proximities in Euclidean embedding ###
M = DX.shape[0]
W = U.dot(np.diag(np.sqrt(sigma)))
EmbX = W[1:M+1, :]
EmbY = W[M+2:, :]
o = W[M+1, :]
DMat = sp.spatial.distance.pdist(W[1:, :]) 
DMat = sp.spatial.distance.squareform(DMat)
# Within sets 
DXe = DMat[:M, :M]
DYe = DMat[M+1:, M+1:]

print("Check theory on distances for the set X", np.all(np.isclose(DXe, np.sqrt(DX**2 + c3*zeta_f) - np.sqrt(np.diag(c3*zeta_f*np.ones(DX.shape[0])))))) # diagonal elements of distance matrices are always 0s
print("Check theory on distances for the set Y", np.all(np.isclose(DYe, np.sqrt(DY**2 + c3*zeta_f) - np.sqrt(np.diag(c3*zeta_f*np.ones(DY.shape[0])))))) # diagonal elements of distance matrices are always 0s
print("Check theory on distance to orgin for the set X:", np.all(np.isclose(DMat[M, :M], np.sqrt(UX**2 + c3*zeta_f))))
print("Check theory on distance to orgin for the set Y:", np.all(np.isclose(DMat[M, M+1:], np.sqrt(UY**2 + c3*zeta_f))))

# Between sets
fXYe = DMat[:M, M+1:]
print("Check theory on distance between points of sets:", np.all(np.isclose(fXYe, np.sqrt(fXY**2 + c3*zeta_f))))


Check theory on distances for the set X False
Check theory on distances for the set Y False
Check theory on distance to orgin for the set X: False
Check theory on distance to orgin for the set Y: False
Check theory on distance between points of sets: False
